# NYC Tree Data Cleaning

### In this notebook, we'll attempt to clean as much as possible the '2015 Street Tree Census - Tree Data' dataset. We have a sample of 100.000 rows instead of the original 684.000 rows.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data_100000.csv')
df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   created_at  100000 non-null  object 
 1   tree_id     100000 non-null  int64  
 2   block_id    100000 non-null  int64  
 3   the_geom    100000 non-null  object 
 4   tree_dbh    100000 non-null  int64  
 5   stump_diam  100000 non-null  int64  
 6   curb_loc    100000 non-null  object 
 7   status      100000 non-null  object 
 8   health      95007 non-null   object 
 9   spc_latin   95008 non-null   object 
 10  spc_common  95008 non-null   object 
 11  steward     95008 non-null   object 
 12  guards      95008 non-null   object 
 13  sidewalk    95008 non-null   object 
 14  user_type   100000 non-null  object 
 15  problems    95008 non-null   object 
 16  root_stone  100000 non-null  object 
 17  root_grate  100000 non-null  object 
 18  root_other  100000 non-null  object 
 19  trn

It seems like we have 4992 rows with some missing values for the columns 'health', 'spc_latin', 'spc_common', 'steward', 'guards', 'problems' and 'sidewalk_damage'. We might me tempted to delete those rows with NaN values but actually all those rows belong to either a 'stump' or 'dead' tree. Thus, it makes sense that they don't have any values for those columns. We should keep them.

In [4]:
df.status.value_counts()

Alive    95008
Stump     2831
Dead      2161
Name: status, dtype: int64

In [5]:
df_alive=df[df['status']=='Alive']
df_alive[df_alive.isnull().any(axis=1)]

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
32794,09/21/2015,245041,413012,POINT (-74.21641202587655 40.54859739520807),16,0,OnCurb,Alive,NaN,Fraxinus pennsylvanica,...,62,24,SI32,Rossville-Woodrow,5020801,New York,40.548597,-74.216412,924106.880773,139219.632044


1 row has the 'health' variable missing.

We only keep the 'latitude' and 'longitude' information concerning the location of each tree.

In [6]:
df = df.drop(['the_geom', 'state', 'zip_city', 'x_sp', 'y_sp', 'address', 'borocode', 'boroname', 'boro_ct', 'cb_num', 'st_assem', 'st_senate', 'nta', 'nta_name', 'cncldist'], 1)

We'll drop all columns not necesarry for machine learning. We will only keep information directly related to the tree.

In [7]:
df = df.drop(['created_at', 'tree_id', 'block_id', 'spc_latin', 'steward', 'guards', 'user_type', 'problems'], 1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   tree_dbh    100000 non-null  int64  
 1   stump_diam  100000 non-null  int64  
 2   curb_loc    100000 non-null  object 
 3   status      100000 non-null  object 
 4   health      95007 non-null   object 
 5   spc_common  95008 non-null   object 
 6   sidewalk    95008 non-null   object 
 7   root_stone  100000 non-null  object 
 8   root_grate  100000 non-null  object 
 9   root_other  100000 non-null  object 
 10  trnk_wire   100000 non-null  object 
 11  trnk_light  100000 non-null  object 
 12  trnk_other  100000 non-null  object 
 13  brnch_ligh  100000 non-null  object 
 14  brnch_shoe  100000 non-null  object 
 15  brnch_othe  100000 non-null  object 
 16  zipcode     100000 non-null  int64  
 17  latitude    100000 non-null  float64
 18  longitude   100000 non-null  float64
dtypes: 

As 'sidewalk' has two values: 'NoDamage' or 'Damage'. I replace it a new one called 'sidewalk_damage' with 'Yes' or 'No'. It seems clearer this way.

In [9]:
def damage(x):
    if x =='NoDamage':
        return 'No'
    elif x =='Damage':
        return 'Yes'
df['sidewalk_damage'] = df['sidewalk'].apply(lambda x: damage(x))
df = df.drop(['sidewalk'], 1)

Let's change the name of some columns so it's clearer to understand what information they contain.

In [10]:
df = df.rename(columns={
    'brnch_othe': 'branch_other',
    'brnch_shoe': 'branch_shoe',
    'brnch_ligh': 'branch_light',
    'trnk_wire': 'trunk_wire',
    'trnk_light': 'trunk_light',
    'trnk_other': 'trunk_other'
    })

I clean the 'spc_common' values (capitalize, replace, ...).

In [14]:
df['spc_common'] = df['spc_common'].str.capitalize().str.replace("-", " ")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tree_dbh         100000 non-null  int64  
 1   stump_diam       100000 non-null  int64  
 2   curb_loc         100000 non-null  object 
 3   status           100000 non-null  object 
 4   health           95007 non-null   object 
 5   spc_common       95008 non-null   object 
 6   root_stone       100000 non-null  object 
 7   root_grate       100000 non-null  object 
 8   root_other       100000 non-null  object 
 9   trunk_wire       100000 non-null  object 
 10  trunk_light      100000 non-null  object 
 11  trunk_other      100000 non-null  object 
 12  branch_light     100000 non-null  object 
 13  branch_shoe      100000 non-null  object 
 14  branch_other     100000 non-null  object 
 15  zipcode          100000 non-null  int64  
 16  latitude         100000 non-null  float